In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import time, os
from time import sleep
from random import randint
import re
import json
from fake_useragent import UserAgent
import math

In [2]:
# url = 'https://www.imdb.com/title/tt1302006/'
# response = requests.get(url)
# page = response.text
# soup = BeautifulSoup(page, "lxml")

In [3]:
def get_movie_crew_and_genre(soup,field_name):
    movie_crew_list = []
    for a in soup.find('h4', text=re.compile(field_name)).parent.find_all('a'):
        movie_crew_list.append(a.text.strip())
    return movie_crew_list

In [4]:
def get_movie_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [5]:
# import dateutil.parser

# def to_date(datestring):
#     date = dateutil.parser.parse(datestring)
#     return date
    

In [11]:
def get_movie_dict(titleId):
    '''
    Grab the titleId 
    '''
    
    url = 'https://www.imdb.com/title/'+str(titleId)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    title_div = soup.find('div',class_='title_wrapper')
    title_div

    IMDB_id = titleId
    
    #title 
    title = get_movie_value(soup,'title')
    title = title.split('(')[0].strip()

    #Get the directors for the movie
    directors = get_movie_crew_and_genre(soup,'Director')


    #get the writers for the movie
    writers = get_movie_crew_and_genre(soup,'Writer')

    #get the cast of the movie:
    casts = get_movie_crew_and_genre(soup,'Stars')
    casts = casts[0:3]

    #genre
    genres = get_movie_crew_and_genre(soup,'Genres')

    #country 
    country = get_movie_crew_and_genre(soup,'Country')
    country = country[0]

    #language
    language = get_movie_crew_and_genre(soup,'Language')
    language = language[0]

    #runtime
    runtime = get_movie_value(soup,'Run')
    try:
        converted_runtime = int(runtime.split()[0])
    except:
        converted_runtime = 0

    #rating
    release_and_rating_string = title_div.find('div',{'class':'subtext'}).text
    rating = release_and_rating_string.split('|')[0].strip()
    rating

    #Release Date
    release_and_rating_string = title_div.find('div',{'class':'subtext'}).text
    release = release_and_rating_string.split('|')[3].strip()
    release = release.split('(')
    release = release[0]
    #converted_release_date = to_date(release_date)

    try:
        budget = soup.find('h4', text=re.compile('Budget')).parent.text
        budget = budget.split('\n')
        budget = budget[1].split('$')
        budget = budget[1].replace(',','')
        budget = int(budget) 
    except:
        budget = math.nan


    #metacritic score
    metacritic_score = soup.find('div',{'class':'titleReviewBar'}).span.text
    try:
        metacritic_score = int(metacritic_score)
    except:
        metacritic_score = math.nan
    
    headers = ['IMDBId','movie title','director(s)','writer(s)','casts','genre(s)','language','country',
           'runtime (mins)', 'mpaarating', 'release date','metacritic score','budget']

    movie_data = []
    movie_dict = dict(zip(headers, [IMDB_id,
                                    title,
                                    directors,
                                    writers,
                                    casts,
                                    genres,
                                    language,
                                    country,
                                    converted_runtime,
                                    rating,
                                    release,
                                    metacritic_score,
                                    budget]))

    #movie_data.append(movie_dict)
    sleep(randint(2,10))
    return movie_dict



In [12]:
get_movie_dict('tt5052448')

{'IMDBId': 'tt5052448',
 'movie title': 'Get Out',
 'director(s)': ['Jordan Peele'],
 'writer(s)': ['Jordan Peele'],
 'casts': ['Daniel Kaluuya', 'Allison Williams', 'Bradley Whitford'],
 'genre(s)': ['Horror', 'Mystery', 'Thriller'],
 'language': 'English',
 'country': 'USA',
 'runtime (mins)': 104,
 'mpaarating': 'R',
 'release date': '24 February 2017 ',
 'metacritic score': 85,
 'budget': 4500000}

In [13]:
# headers = ['IMDBId','movie title','director(s)','writer(s)','casts','genre(s)','language','country',
#            'runtime (mins)', 'rating', 'release date','budget']

# movie_data = []
# movie_dict = dict(zip(headers, [IMDB_id,
#                                 title,
#                                 directors,
#                                 writers,
#                                 casts,
#                                 genres,
#                                 language,
#                                 country,
#                                 converted_runtime,
#                                 converted_release_date,
#                                 rating, 
#                                 budget]))

# movie_data.append(movie_dict)
# movie_data

In [14]:
movie_list = []
links = ['tt0454876',
 'tt0092593',
 'tt7131622',
 'tt0088683',
 'tt0090886',
 'tt0268126',
 'tt0090756',
 'tt1532503',
 'tt0119164',
 'tt0478304']
for link in links:
    movie_list.append(get_movie_dict(link))

In [24]:
import pandas as pd
movies_info_df = pd.DataFrame(movie_list)
movies_info_df.set_index('IMDBId', inplace=True)
movies_info_df

,movie title,director(s),writer(s),casts,genre(s),language,country,runtime (mins),mpaarating,release date,metacritic score,budget
IMDBId,,,,,,,,,,,,
tt0454876,Life of Pi,[Ang Lee],"[Yann Martel, David Magee]","[Suraj Sharma, Irrfan Khan, Adil Hussain]","[Adventure, Drama, Fantasy]",English,USA,127,PG,21 November 2012,79.0,120000000.0
tt0092593,Au Revoir les Enfants,[Louis Malle],"[Louis Malle, Louis Malle]","[Gaspard Manesse, Raphael Fejtö, Francine Race...","[Drama, War]",French,France,104,PG,12 February 1988,88.0,NaN
tt7131622,Once Upon a Time... in Hollywood,[Quentin Tarantino],[Quentin Tarantino],"[Leonardo DiCaprio, Brad Pitt, Margot Robbie]","[Comedy, Drama]",English,USA,161,R,26 July 2019,83.0,90000000.0
tt0088683,Agnes of God,[Norman Jewison],"[John Pielmeier, John Pielmeier]","[Jane Fonda, Anne Bancroft, Meg Tilly]","[Drama, Mystery]",English,USA,98,PG-13,27 September 1985,52.0,10000000.0
tt0090886,Crimes of the Heart,[Bruce Beresford],"[Beth Henley, Beth Henley]","[Diane Keaton, Jessica Lange, Sissy Spacek]","[Comedy, Drama]",English,USA,105,PG-13,23 January 1987,NaN,20000000.0
tt0268126,Adaptation.,[Spike Jonze],"[Susan Orlean, Charlie Kaufman]","[Nicolas Cage, Meryl Streep, Chris Cooper]","[Comedy, Drama]",English,USA,115,R,14 February 2003,83.0,19000000.0
tt0090756,Blue Velvet,[David Lynch],[David Lynch],"[Isabella Rossellini, Kyle MacLachlan, Dennis ...","[Drama, Mystery, Thriller]",English,USA,120,R,23 October 1986,76.0,6000000.0
tt1532503,Beginners,[Mike Mills],[Mike Mills],"[Ewan McGregor, Christopher Plummer, Mélanie L...","[Comedy, Drama, Romance]",English,USA,105,R,9 June 2011,81.0,3200000.0
tt0119164,The Full Monty,[Peter Cattaneo],[Simon Beaufoy],"[Robert Carlyle, Tom Wilkinson, Mark Addy]","[Comedy, Drama]",English,UK,91,R,19 September 1997,75.0,3500000.0


In [26]:
movies_info_df[['writer1','writer2']] = pd.DataFrame(movies_info_df['writer(s)'].tolist(), index= movies_info_df.index)

In [27]:
movies_info_df

,movie title,director(s),writer(s),casts,genre(s),language,country,runtime (mins),mpaarating,release date,metacritic score,budget,writer1,writer2
IMDBId,,,,,,,,,,,,,,
tt0454876,Life of Pi,[Ang Lee],"[Yann Martel, David Magee]","[Suraj Sharma, Irrfan Khan, Adil Hussain]","[Adventure, Drama, Fantasy]",English,USA,127,PG,21 November 2012,79.0,120000000.0,Yann Martel,David Magee
tt0092593,Au Revoir les Enfants,[Louis Malle],"[Louis Malle, Louis Malle]","[Gaspard Manesse, Raphael Fejtö, Francine Race...","[Drama, War]",French,France,104,PG,12 February 1988,88.0,NaN,Louis Malle,Louis Malle
tt7131622,Once Upon a Time... in Hollywood,[Quentin Tarantino],[Quentin Tarantino],"[Leonardo DiCaprio, Brad Pitt, Margot Robbie]","[Comedy, Drama]",English,USA,161,R,26 July 2019,83.0,90000000.0,Quentin Tarantino,None
tt0088683,Agnes of God,[Norman Jewison],"[John Pielmeier, John Pielmeier]","[Jane Fonda, Anne Bancroft, Meg Tilly]","[Drama, Mystery]",English,USA,98,PG-13,27 September 1985,52.0,10000000.0,John Pielmeier,John Pielmeier
tt0090886,Crimes of the Heart,[Bruce Beresford],"[Beth Henley, Beth Henley]","[Diane Keaton, Jessica Lange, Sissy Spacek]","[Comedy, Drama]",English,USA,105,PG-13,23 January 1987,NaN,20000000.0,Beth Henley,Beth Henley
tt0268126,Adaptation.,[Spike Jonze],"[Susan Orlean, Charlie Kaufman]","[Nicolas Cage, Meryl Streep, Chris Cooper]","[Comedy, Drama]",English,USA,115,R,14 February 2003,83.0,19000000.0,Susan Orlean,Charlie Kaufman
tt0090756,Blue Velvet,[David Lynch],[David Lynch],"[Isabella Rossellini, Kyle MacLachlan, Dennis ...","[Drama, Mystery, Thriller]",English,USA,120,R,23 October 1986,76.0,6000000.0,David Lynch,None
tt1532503,Beginners,[Mike Mills],[Mike Mills],"[Ewan McGregor, Christopher Plummer, Mélanie L...","[Comedy, Drama, Romance]",English,USA,105,R,9 June 2011,81.0,3200000.0,Mike Mills,None
tt0119164,The Full Monty,[Peter Cattaneo],[Simon Beaufoy],"[Robert Carlyle, Tom Wilkinson, Mark Addy]","[Comedy, Drama]",English,UK,91,R,19 September 1997,75.0,3500000.0,Simon Beaufoy,None


In [30]:
pd.get_dummies(list(movies_info_df['writer1']))

,Beth Henley,David Lynch,John Pielmeier,Louis Malle,Mike Mills,Quentin Tarantino,Simon Beaufoy,Susan Orlean,Terrence Malick,Yann Martel
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0
6,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,1,0
